In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
import matplotlib

%matplotlib inline

In [2]:
P = np.array([  [1, -1/2], 
                [-1/2, 2]])

q = np.array([-1, 0])

x = cp.Variable(2)

obj = cp.quad_form(x,P) + q.T @ x
u = cp.Parameter(2)
A = np.array([  [1,2],
                [1,-4],
                [1,1]])

constraints = [
    A[0] @ x <= u[0],
    A[1] @ x <= u[1],
    A[2] @ x >= -5
]

prob = cp.Problem(cp.Minimize(obj), constraints=constraints)


In [3]:
# (a)
u.value = np.array([-2,-3])
prob.solve()
assert prob.status == cp.OPTIMAL
print('prob value:', prob.value)
lS = np.array([c.dual_value for c in constraints])
xS = np.array(x.value)
print('primal optimal', xS)
print('dual optimal:', lS)

prob value: 8.222222222222223
primal optimal [-2.33333333  0.16666667]
dual optimal: [3.38888889 2.44444444 0.        ]


In [4]:
# Check KKT
print('Checking KKT')
# primal feasible
assert np.array([c.value() for c in constraints]).all()
# dual feasible
assert (lS >= 0).all()
# comp. slackness
f = [c.expr.value for c in constraints]
assert np.isclose(lS @ f , 0)
# grad_x L(xS,lS) = 0
L_grad = 2*(P @ xS) + q + A.T @ lS
print('L grad', L_grad)
assert np.isclose(L_grad, 0).all()
print('KKT ok')

Checking KKT
L grad [0. 0.]
KKT ok


In [5]:
print('x2:', round(xS[1], 2))
print('lambda3:', round(lS[2],2))

x2: 0.17
lambda3: 0.0


In [6]:
# (b)
print('Pertubation table')

delta = [0, -0.1 ,0.1]
p_org = prob.value
dV = []
for d1 in delta:
    for d2 in delta:
        u.value = np.array([-2 + d1, -3 + d2])
        p_exact = prob.solve()
        assert prob.status == cp.OPTIMAL
        p_pred = p_org - lS.T @ [d1,d2,0]
        assert p_pred <= p_exact
        print(f'd1 {d1} d2 {d2} p_pred {p_pred} p_exact {p_exact} '
                f'exact - pred:{p_exact - p_pred}'
            )
        dV.append((p_exact - p_pred))

print('diff argsort', np.argsort(dV))

Pertubation table
d1 0 d2 0 p_pred 8.222222222222223 p_exact 8.222222222222229 exact - pred:5.329070518200751e-15
d1 0 d2 -0.1 p_pred 8.466666666666669 p_exact 8.468885979233272 exact - pred:0.0022193125666039037
d1 0 d2 0.1 p_pred 7.977777777777779 p_exact 7.980005615605602 exact - pred:0.0022278378278235422
d1 -0.1 d2 0 p_pred 8.561111111111112 p_exact 8.565001721506773 exact - pred:0.003890610395661298
d1 -0.1 d2 -0.1 p_pred 8.805555555555557 p_exact 8.815552599287349 exact - pred:0.009997043731791777
d1 -0.1 d2 0.1 p_pred 8.316666666666668 p_exact 8.318894597656488 exact - pred:0.002227930989819882
d1 0.1 d2 0 p_pred 7.883333333333335 p_exact 7.887210584052887 exact - pred:0.003877250719552805
d1 0.1 d2 -0.1 p_pred 8.127777777777778 p_exact 8.129997137000323 exact - pred:0.0022193592225452363
d1 0.1 d2 0.1 p_pred 7.638888888888889 p_exact 7.648894411332476 exact - pred:0.010005522443586656
diff argsort [0 1 7 2 5 6 3 4 8]
